In [ ]:
import sys
from google.colab import drive
drive.mount('/content/drive')
sys.path.append(r"/content/drive/MyDrive/")

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

In [ ]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from loader_util.nn.conv import MinVGGNet

##
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras.datasets import cifar10

In [ ]:
# the argument parser
args = {
    "output": r"/content/drive/MyDrive/TrainingFolder/p6_ensemble/output",
    "models": r"/content/drive/MyDrive/TrainingFolder/p6_ensemble/models",
    "num_models": 5
}

In [ ]:
(trainx, trainy), (testx, testy) = cifar10.load_data()
trainx = trainx.astype("float") / 255.0
testx = testx.astype("float") / 255.0

# encode the labels
lb = LabelBinarizer()
trainy = lb.fit_transform(trainy)
testy = lb.transform(testy)

# initialize the label names for the CIFAR-10 dataset
labelNames = ["airplane", "automobile", "bird", "cat", "deer",
              "dog", "frog", "horse", "ship", "truck"]

In [ ]:
# construct data augmentor
aug = ImageDataGenerator(rotation_range=10,
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         horizontal_flip=True,
                         fill_mode='nearest')

In [ ]:
for i in range(args["num_models"]):
    # init the optimizer and model
    print(f"[INFO] training model {i+1}/{args['num_models']}......")
    opt = SGD(lr=0.01, decay=0.01/40, momentum=0.9, nesterov=True)
    model = MinVGGNet.build(width=32, height=32, depth=3, classes=len(labelNames))
    model.compile(loss="categorical_crossentropy",
                  optimizer=opt,
                  metrics=["accuracy"])
    
    # train the network
    H = model.fit_generator(aug.flow(trainx, trainy, batch_size=64),
                            validation_data=(testx, testy),
                            epochs=40, verbose=1)
    # save the model
    p = [args["models"], f"model_{i}.model"]
    model.save(os.path.sep.join(p))
    
    # eval the netowrk
    # evaluate the network
    predictions = model.predict(testx, batch_size=32 )
    report = classification_report(testy.argmax(axis=1 ),
    predictions.argmax(axis=1 ), target_names=labelNames)
    
    # save the report
    p = [args["output"], f"model_{i}.txt"]
    f = open(os.path.sep.join(p), "w")
    f.write(report)
    f.close()
    
    # save the plots

    # plot the performance
    epochs = range(1,41)
    loss = H.history['loss']
    accuracy = H.history['accuracy']
    val_loss = H.history['val_loss']
    val_accuracy = H.history['val_accuracy']
    plot_df = pd.DataFrame(data=np.c_[epochs,loss,accuracy,val_loss,val_accuracy], 
                           columns=['epochs','loss', 'accuracy', 'val_loss', 'val_accuracy'])
    
    # do the actual plots
    p = [args["output"], f"model_{i}.png"]
    sns.set(font_scale=1)
    f, ax = plt.subplots(1, 1, figsize=(15,8))
    sns.lineplot(data=plot_df, x='epochs', y='loss', ax=ax, label='train loss', linewidth=3)
    sns.lineplot(data=plot_df, x='epochs', y='accuracy', ax=ax, label='train accuracy', linewidth=3)
    sns.lineplot(data=plot_df, x='epochs', y='val_loss', ax=ax, label='val loss', linewidth=3)
    sns.lineplot(data=plot_df, x='epochs', y='val_accuracy', ax=ax, label='val_accuracy', linewidth=3)
    ax.set_ylabel('Loss or Accuracy')
    ax.set_xlabel('Epochs')
    plt.setp(ax.get_legend().get_texts(), fontsize='18'); # for legend text
    plt.savefig(os.path.sep.join(p))
    plt.close()
    

[INFO] training model 1/5......


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
782/782 [==============================] - 28s 35ms/step - loss: 1.8103 - accuracy: 0.3944 - val_loss: 1.4556 - val_accuracy: 0.5071
Epoch 2/40
782/782 [==============================] - 27s 34ms/step - loss: 1.3697 - accuracy: 0.5197 - val_loss: 1.1332 - val_accuracy: 0.6023
Epoch 3/40
782/782 [==============================] - 27s 35ms/step - loss: 1.1850 - accuracy: 0.5807 - val_loss: 1.0516 - val_accuracy: 0.6300
Epoch 4/40
782/782 [==============================] - 27s 34ms/step - loss: 1.0797 - accuracy: 0.6204 - val_loss: 0.9384 - val_accuracy: 0.6749
Epoch 5/40
782/782 [==============================] - 27s 34ms/step - loss: 1.0048 - accuracy: 0.6445 - val_loss: 0.9124 - val_accuracy: 0.6802
Epoch 6/40
782/782 [==============================] - 27s 34ms/step - loss: 0.9453 - accuracy: 0.6673 - val_loss: 0.8554 - val_accuracy: 0.7106
Epoch 7/40
782/782 [==============================] - 27s 34ms/step - loss: 0.9037 - accuracy: 0.6818 - val_loss: 0.8038 - val_accuracy:

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
782/782 [==============================] - 28s 35ms/step - loss: 1.7549 - accuracy: 0.4088 - val_loss: 1.4164 - val_accuracy: 0.5062
Epoch 2/40
782/782 [==============================] - 27s 34ms/step - loss: 1.3278 - accuracy: 0.5314 - val_loss: 1.2199 - val_accuracy: 0.5808
Epoch 3/40
782/782 [==============================] - 27s 34ms/step - loss: 1.1512 - accuracy: 0.5937 - val_loss: 1.0396 - val_accuracy: 0.6409
Epoch 4/40
782/782 [==============================] - 27s 34ms/step - loss: 1.0471 - accuracy: 0.6304 - val_loss: 0.9342 - val_accuracy: 0.6869
Epoch 5/40
782/782 [==============================] - 27s 35ms/step - loss: 0.9751 - accuracy: 0.6574 - val_loss: 0.9470 - val_accuracy: 0.6773
Epoch 6/40
782/782 [==============================] - 27s 35ms/step - loss: 0.9264 - accuracy: 0.6735 - val_loss: 0.8050 - val_accuracy: 0.7160
Epoch 7/40
782/782 [==============================] - 27s 34ms/step - loss: 0.8970 - accuracy: 0.6850 - val_loss: 0.7754 - val_accuracy:

...

Epoch 1/40
782/782 [==============================] - 28s 35ms/step - loss: 1.7684 - accuracy: 0.4056 - val_loss: 1.5123 - val_accuracy: 0.4847
Epoch 2/40
782/782 [==============================] - 27s 34ms/step - loss: 1.3358 - accuracy: 0.5330 - val_loss: 1.2561 - val_accuracy: 0.5658
Epoch 3/40
782/782 [==============================] - 27s 35ms/step - loss: 1.1741 - accuracy: 0.5882 - val_loss: 1.2971 - val_accuracy: 0.5666
Epoch 4/40
782/782 [==============================] - 27s 34ms/step - loss: 1.0718 - accuracy: 0.6210 - val_loss: 0.9451 - val_accuracy: 0.6760
Epoch 5/40
782/782 [==============================] - 27s 35ms/step - loss: 1.0005 - accuracy: 0.6461 - val_loss: 0.9264 - val_accuracy: 0.6830
Epoch 6/40
782/782 [==============================] - 27s 34ms/step - loss: 0.9390 - accuracy: 0.6679 - val_loss: 0.9053 - val_accuracy: 0.6897
Epoch 7/40
782/782 [==============================] - 27s 34ms/step - loss: 0.9050 - accuracy: 0.6794 - val_loss: 0.8492 - val_accuracy:

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
782/782 [==============================] - 28s 35ms/step - loss: 1.7795 - accuracy: 0.4056 - val_loss: 1.4706 - val_accuracy: 0.5044
Epoch 2/40
782/782 [==============================] - 27s 34ms/step - loss: 1.3540 - accuracy: 0.5243 - val_loss: 1.1444 - val_accuracy: 0.5952
Epoch 3/40
782/782 [==============================] - 27s 34ms/step - loss: 1.1891 - accuracy: 0.5821 - val_loss: 1.1571 - val_accuracy: 0.6153
Epoch 4/40
782/782 [==============================] - 27s 35ms/step - loss: 1.0729 - accuracy: 0.6220 - val_loss: 0.9515 - val_accuracy: 0.6794
Epoch 5/40
782/782 [==============================] - 27s 35ms/step - loss: 0.9958 - accuracy: 0.6477 - val_loss: 1.2507 - val_accuracy: 0.6130
Epoch 6/40
782/782 [==============================] - 27s 35ms/step - loss: 0.9437 - accuracy: 0.6686 - val_loss: 1.0546 - val_accuracy: 0.6622
Epoch 7/40
782/782 [==============================] - 27s 35ms/step - loss: 0.9040 - accuracy: 0.6825 - val_loss: 0.8181 - val_accuracy:

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
782/782 [==============================] - 28s 35ms/step - loss: 1.7688 - accuracy: 0.4029 - val_loss: 1.6205 - val_accuracy: 0.4606
Epoch 2/40
782/782 [==============================] - 27s 34ms/step - loss: 1.3392 - accuracy: 0.5296 - val_loss: 1.1127 - val_accuracy: 0.6134
Epoch 3/40
782/782 [==============================] - 26s 34ms/step - loss: 1.1663 - accuracy: 0.5888 - val_loss: 1.1419 - val_accuracy: 0.6192
Epoch 4/40
782/782 [==============================] - 27s 34ms/step - loss: 1.0633 - accuracy: 0.6270 - val_loss: 1.1202 - val_accuracy: 0.6311
Epoch 5/40
782/782 [==============================] - 27s 34ms/step - loss: 0.9925 - accuracy: 0.6517 - val_loss: 0.8880 - val_accuracy: 0.6940
Epoch 6/40
782/782 [==============================] - 27s 35ms/step - loss: 0.9448 - accuracy: 0.6657 - val_loss: 0.9844 - val_accuracy: 0.6789
Epoch 7/40
782/782 [==============================] - 27s 34ms/step - loss: 0.9021 - accuracy: 0.6809 - val_loss: 0.8677 - val_accuracy: